# Parse images into input vector from training set

In [68]:
import numpy as np
import os

dataPath = 'RealImageNet/ImageNetSubsample/Data/CLS-LOC'
trainPath = os.path.join(dataPath, 'train')
testPath = os.path.join(dataPath, 'test')
validationPath = os.path.join(dataPath, 'val')



In [83]:
from keras.preprocessing import image
# get training data
x_train = []
y_train = []
trainingFolders = [x for x in os.listdir(trainPath)]

lines = [line.rstrip('\n').split() for line in open('RealImageNet/LOC_synset_mapping.txt')]
# print(lines)
wnids_to_words = {line[0]:' '.join(line[1:]) for line in lines }

train_bboxes = {} #{image_name: list(bboxes)}
for id in trainingFolders:
    boxesPath = os.path.join("RealImageNet", "LOC_train_solution.csv")
    lines = [line.rstrip('\n').split(',') for line in open(boxesPath)][1:]
    for line in lines:
        imageId = line[0]
        predictionString = line[1]
        split = predictionString.split()
        train_bboxes[imageId] = []
        for i in range(0, len(split), 5):
            box = split[i:i+5]
            train_bboxes[imageId].append(box)

for imageName in train_bboxes.keys():
    imageLabel = imageName.split('_')[0]
    imagePath = os.path.join(trainPath, imageLabel, imageName + ".JPEG") #folder name
    
    if os.path.exists(imagePath):
        img = image.load_img(imagePath, target_size=(224, 224)) #pil format
        x = image.img_to_array(img) 
#         x = np.expand_dims(x, axis=0) # making a numpy array (surrounds with another lis)
        x_train.append(x/255.)
        y_train.append(float(imageLabel[1:]))

x_train = np.array(x_train)
y_train = np.array(y_train)


from sklearn.preprocessing import LabelBinarizer
label_encoder = LabelBinarizer()
y_train = label_encoder.fit_transform(y_train)

# print(transfomed_label)


In [84]:
y_train

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0]])

In [17]:
# get validation data

validation_bboxes = {} #{image_name: list(bboxes)}
# for id in trainingFolders:
valBoxesPath = os.path.join("RealImageNet", "LOC_val_solution.csv")
lines = [line.rstrip('\n').split(',') for line in open(valBoxesPath)][1:]
for line in lines:
    imageId = line[0]
    predictionString = line[1]
    split = predictionString.split()
    validation_bboxes[imageId] = []
    for i in range(0, len(split), 5):
        box = split[i:i+5]
        validation_bboxes[imageId].append(box)

for imageName in validation_bboxes.keys():
    imagePath = os.path.join(validationPath, imageName + ".JPEG")
    if os.path.exists(imagePath):
        img = image.load_img(imagePath, target_size=(224, 224)) #pil format
        x = image.img_to_array(img)  


# get test data
    
            

[[[ 19.  11.   0.]
  [ 20.  13.   3.]
  [ 22.  13.   4.]
  ...
  [ 66.  32.  22.]
  [ 52.  41.  23.]
  [ 44.  38.  22.]]

 [[ 20.  11.   2.]
  [ 25.  12.   4.]
  [ 34.  14.   5.]
  ...
  [ 60.  38.  27.]
  [ 50.  36.  23.]
  [ 37.  27.  15.]]

 [[ 24.  16.   5.]
  [ 26.  13.   4.]
  [ 31.  17.   8.]
  ...
  [ 16.  13.   8.]
  [ 37.  23.  10.]
  [ 49.  35.  22.]]

 ...

 [[121. 123. 109.]
  [ 78.  89.  83.]
  [ 83.  82.  87.]
  ...
  [ 81.  72.  67.]
  [ 61.  68.  74.]
  [100. 102.  99.]]

 [[ 74.  89.  86.]
  [ 84.  99.  92.]
  [102. 103.  97.]
  ...
  [ 56.  57.  61.]
  [ 64.  65.  70.]
  [ 52.  51.  57.]]

 [[ 97. 101. 100.]
  [105. 101. 100.]
  [ 82.  90.  92.]
  ...
  [119. 116. 101.]
  [ 63.  73.  64.]
  [ 52.  63.  59.]]]
[[[194. 191. 198.]
  [192. 195. 202.]
  [200. 190. 199.]
  ...
  [211. 217. 213.]
  [215. 216. 208.]
  [217. 216. 214.]]

 [[197. 192. 198.]
  [201. 190. 198.]
  [193. 192. 197.]
  ...
  [216. 217. 211.]
  [220. 216. 215.]
  [224. 218. 218.]]

 [[192. 196. 197.]

# Set up and train model 

In [88]:
# from keras.models import Sequential # How layers interact (x -> y)
# from keras.layers import Dense      # Type of layer (what you feed your data into)

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

img_width, img_height = 224, 224

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model = Sequential()
# model.add(Dense(units=256, activation="softmax"))

# # LATER: Specify input shape!!

# #define metrics, loss function (maybe our own?), gradient descent
# model.compile(loss = "categorical_crossentropy", metrics = ["accuracy"], optimizer = "adam")

# # Train model
model.fit(x_train, y_train, epochs = 1, verbose=1)

# Validate

# Evaluate


Epoch 1/1
2589/2589 [==============================] - 132s 51ms/step - loss: 0.3962 - acc: 0.8225


In [ ]:
from keras.preprocessing import image
import numpy as np

# LATER: Add the image net parsing here, then do for-loop
# 



# img_path = 'elephant.jpg'
# img = image.load_img(img_path, target_size=(224, 224)) # kil format based on pillow -> loads to certain size
# x = image.img_to_array(img)                            
# x = np.expand_dims(x, axis=0)                          # making a numpy array

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

# Train a model to classify

In [92]:
pred = model.predict(x_train[1000:2000])

In [93]:
strs = ['n0' + str(int(x)) for x in label_encoder.inverse_transform(pred)]
outputs = [wnids_to_words[x] for x in strs]

In [94]:
outputs

['great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'tench, Tinca tinca',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'tench, Tinca tinca',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias',
 'great white shark, white shark, man-eater, man-eating shark, Carcharodon

In [95]:
pred

array([[1.2800553e-01, 4.5720484e-02, 3.6871493e-01, 3.0535096e-01,
        2.8525916e-01],
       [4.2076674e-01, 2.0660526e-01, 1.3744439e-01, 1.6599910e-01,
        2.9177552e-01],
       [5.9552066e-02, 7.1208037e-02, 3.9570114e-01, 3.6384070e-01,
        3.4762844e-01],
       ...,
       [3.5772937e-05, 9.9976665e-01, 9.0615809e-05, 4.1481742e-04,
        7.5554465e-07],
       [1.3018198e-05, 9.9997962e-01, 2.5663362e-06, 2.7123282e-05,
        1.4875901e-08],
       [4.9228752e-06, 9.9998188e-01, 1.9305198e-05, 1.4862674e-04,
        2.3894129e-08]], dtype=float32)